# Вебинар 3. Коллаборативная фильтрация

In [1]:
pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.3 MB/s eta 0:00:01
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-linux_x86_64.whl size=4716920 sha256=d7bb23934ff133f4a2084ce438f06ff984897610f8970d536ec7ed9bc95e39b2
  Stored in directory: /home/aleks/.cache/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
Successfully built implicit
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [3]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [5]:
# df_popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

In [4]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# ALS

In [11]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 19.7 s, sys: 20.2 s, total: 40 s
Wall time: 5.81 s


In [12]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [13]:
[id_to_itemid[rec[0]] for rec in recs]

[5569230, 999999, 1133018, 1082185, 1106523]

In [14]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [15]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 1min 18s, sys: 2min 17s, total: 3min 36s
Wall time: 28.7 s


0.15770969362129583

In [16]:
result.head(2)

,user_id,actual,als
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1033142, 878996, 995242, 999999, 1005186]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[951590, 1106523, 5569327, 1133018, 1022003]"


In [18]:
regularizations = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
factors = [10, 50, 100, 250, 500]
iterations = [10, 15, 20, 30]

In [19]:
%%time
for reg in regularizations:
    for fact in factors:
        for iters in iterations:
            model = AlternatingLeastSquares(
                factors=fact,
                regularization=reg,
                iterations=iters, 
                calculate_training_loss=True, 
                num_threads=4
            )
            model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
            recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)
            result[f'regularization: {reg}, factors: {fact}, iterations: {iters}'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 1h 51min 52s, sys: 2h 9min 53s, total: 4h 1min 46s
Wall time: 32min 25s


<timed exec>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [20]:
result.head(2)

,user_id,actual,als,"regularization: 0.001, factors: 10, iterations: 10","regularization: 0.001, factors: 10, iterations: 15","regularization: 0.001, factors: 10, iterations: 20","regularization: 0.001, factors: 10, iterations: 30","regularization: 0.001, factors: 50, iterations: 10","regularization: 0.001, factors: 50, iterations: 15","regularization: 0.001, factors: 50, iterations: 20",...,"regularization: 1000, factors: 100, iterations: 20","regularization: 1000, factors: 100, iterations: 30","regularization: 1000, factors: 250, iterations: 10","regularization: 1000, factors: 250, iterations: 15","regularization: 1000, factors: 250, iterations: 20","regularization: 1000, factors: 250, iterations: 30","regularization: 1000, factors: 500, iterations: 10","regularization: 1000, factors: 500, iterations: 15","regularization: 1000, factors: 500, iterations: 20","regularization: 1000, factors: 500, iterations: 30"
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1033142, 878996, 995242, 999999, 1005186]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 1029743, 6534178, 995242]","[999999, 1082185, 1029743, 6534178, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[5569230, 999999, 1106523, 1082185, 1133018]","[5569230, 999999, 1082185, 1133018, 1106523]","[5569230, 999999, 1133018, 1106523, 1082185]",...,"[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[951590, 1106523, 5569327, 1133018, 1022003]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 1029743, 6534178, 995242]","[999999, 1082185, 1029743, 6534178, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[5569230, 999999, 1106523, 1082185, 1133018]","[5569230, 999999, 1082185, 1133018, 1106523]","[5569230, 999999, 1133018, 1106523, 1082185]",...,"[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]"


In [21]:
model_names = [m for m in list(result)[2:] ]

In [23]:
k = 5
res_table = pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)

In [24]:
res_table

,model name,precision@5
0,als,0.157710
1,"regularization: 0.001, factors: 10, iterations...",0.185635
2,"regularization: 0.001, factors: 10, iterations...",0.185635
3,"regularization: 0.001, factors: 10, iterations...",0.185635
4,"regularization: 0.001, factors: 10, iterations...",0.185635
...,...,...
136,"regularization: 1000, factors: 250, iterations...",0.185635
137,"regularization: 1000, factors: 500, iterations...",0.185635
138,"regularization: 1000, factors: 500, iterations...",0.185635
139,"regularization: 1000, factors: 500, iterations...",0.185635


In [25]:
sorted_table = res_table.sort_values(f'precision@{k}', ascending=False)

In [26]:
sorted_table

,model name,precision@5
140,"regularization: 1000, factors: 500, iterations...",0.185635
63,"regularization: 1, factors: 10, iterations: 20",0.185635
116,"regularization: 100, factors: 250, iterations: 30",0.185635
115,"regularization: 100, factors: 250, iterations: 20",0.185635
114,"regularization: 100, factors: 250, iterations: 15",0.185635
...,...,...
77,"regularization: 1, factors: 500, iterations: 10",0.035359
20,"regularization: 0.001, factors: 500, iteration...",0.034254
40,"regularization: 0.01, factors: 500, iterations...",0.034254
60,"regularization: 0.1, factors: 500, iterations: 30",0.023707


In [27]:
sorted_table.iloc[0]['model name']

'regularization: 1000, factors: 500, iterations: 30'

Дальше скора 0.185635 пробиться не получилось. Оптимальные параметры:

regularization: 0.001, factors: 10, iterations: 10'

Видно, что изменения параметра factors, требует изменения регуляризации и количества итераций (увеличения), для достижения оптимыльного скора.

-----

# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель) - Уточнить задачу у менеджера сколько % хочет видеть и за какой срок, а может вообще задача для удержания оттока клиента

- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)-После постановки задачи, проанализировать результаты, учитывая бюджет и т.д
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они? - Если и деалть, то для пользователей которые давно не пользовались услугой, чтобы напомнить о себе. Либо для пользователей которые только что приобрели товар. Но точно не стал бы часто закидывать спам рекомендациями потому что это сильно раздражает пользователя, и вероятность отписки от рассылки очень большая
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе? - От СМС точно отказаться, такая рассылка дорого будет обходиться и как правило игнорируется многими пользователями. На чеках просто не будет обращать внимания, а если даже и какой-то процент и увидит, без визуальной оценки товара интереса не привлечет. Единсвенное я бы предложил пользоваться СМС и чеком только если будут громкие акции на какую-нибудь категорию(Но та как это затратно, нужно смотреть бюджет) Самое эффективное на мой взгляд из этих вариантов это push-уведомления, причем они не должны вылезать часто, чтобы так же не раздрадать пользователя 
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?) - Это зависит от того, на каком устройстве просматривается e-mail, и в любом случае должно быть меньше 10, нельзя заваливать пользоваетля рекомендациями. Я бы разбил рекомендации на категории часть товаров топ лист похожих от последней покупки, и товары которые были чаще просмотренны или оценены.
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки - Тут мне самому интересно)
- Cколько мы хотим зарабатывать с одного привлеченного юзера? - Тут мне самому интересно)
---
- А точно нужно сортировать по вероятности? - Профитнее собрать и look-alike и категории и вероятность, скорее всего это дорого и трудозатратно, но и профит от этого больше гораздо.
- Какую метрику использовать? - Это только после ТЗ, и возможно даже после EDA датасета. Мне сложно с ходу оценить
- Сколько раз в неделю отпрпавляем рассылку? Рассылку можно отправлять в начале недели и перед выходными, а так же когда есть акции
- В какое время отправляем рассылку? с 18-19 по мск, когда заканчивается рабочий день и люди едут домой
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались? - Нам поможет сэмплирование
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? - Как добиться того, чтобы они были разными? - Я думаю оптимально иметь в одной рассылке разнообразные товары, делить их по категориям, либо как писал выше, учитывая взаимодействие пользователя.
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [ ]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))
data_train['price'].max()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


499.99

In [ ]:
# < 1$
data_train['price'].quantile(0.20)

0.99

In [ ]:
# > 100$
data_train['price'].quantile(0.99995)

84.8129592499882

In [1]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    df_popularity = data_train.groupby('item_id')['user_id'].nunique() / data_train['user_id'].nunique()
    df_popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товарыs
    
    # ...
    
def postfilter_items(user_id, recommednations):
    pass

Все эти функции отправим затем в *utils.py*

# ДЗ

## Теория:

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

## Практика:

4) Поэкспериментировать с ALS (grid-search) ********
   * Подобрать лучшие параметры
   * Написать отчет об увиденном

# Links

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/